In [1]:
!pip install qiskit qiskit-aer matplotlib

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel, depolarizing_error
import numpy as np
import matplotlib.pyplot as plt

def inverse_qft(qc, n):
    """Apply the inverse Quantum Fourier Transform on n qubits."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        qc.h(j)
        for k in range(j):
            qc.cp(-np.pi / 2 ** (j - k), k, j)
    return qc



In [3]:
def qpe_circuit(num_count_qubits, theta, visualize=False):
    qc = QuantumCircuit(num_count_qubits + 1, num_count_qubits)

    # Step 1: Hadamard on counting qubits
    qc.h(range(num_count_qubits))

    # Step 2: Prepare eigenstate |1>
    qc.x(num_count_qubits)

    # Step 3: Controlled-U (phase gate) scaling 2^k
    for qubit in range(num_count_qubits):
        qc.cp(2 * np.pi * theta * (2 ** qubit), qubit, num_count_qubits)

    # Step 4: Inverse QFT
    inverse_qft(qc, num_count_qubits)

    # Step 4.1: Visualize circuit before measurement (Task 4)
    if visualize:
        qc.draw("mpl")  # No errors for mpl

    # Step 5: Measurement
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    return qc

In [7]:
#  COMPARISON WITH THEORY (TASK 3)
def compare_with_theory(counts, theta, num_count_qubits):
    expected_decimal = round(theta * 2**num_count_qubits)
    expected_binary = format(expected_decimal, f"0{num_count_qubits}b")
    measured_binary = max(counts, key=counts.get)

    print("\n=== Comparison with Theory ===")
    print(f"Theoretical θ = {theta}")
    print(f"Expected decimal = {expected_decimal}")
    print(f"Expected binary ({num_count_qubits} bits) = {expected_binary}")
    print(f"Measured most frequent output = {measured_binary}")

    if measured_binary == expected_binary:
        print("✅ Match — Simulation agrees with theory.")
    else:
        print(" Does NOT match — due to noise / insufficient qubits.")


In [8]:
def run_qpe(num_count_qubits=3, theta=0.125, noisy=False, visualize=False):
    simulator = AerSimulator()

    # Task 5 – Noise Simulation
    if noisy:
        noise_model = NoiseModel()
        noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01, 1), ["h", "x"])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(0.02, 2), ["cp"])
    else:
        noise_model = None

    qc = qpe_circuit(num_count_qubits, theta, visualize)
    compiled = transpile(qc, simulator)

    result = simulator.run(compiled, shots=2048, noise_model=noise_model).result()
    counts = result.get_counts()

    # Show histogram
    plot_histogram(counts)
    plt.show()

    # Show circuit text output
    print(qc.draw(output="text"))

    # Compare theory vs simulation (Task 3)
    compare_with_theory(counts, theta, num_count_qubits)

    return counts

In [9]:
print("\n=== TASK 1: Change Phase Value ===")
run_qpe(num_count_qubits=3, theta=0.25)

print("\n=== TASK 2: Increase Counting Qubits ===")
run_qpe(num_count_qubits=5, theta=0.125)

print("\n=== TASK 3: Compare With Theory ====")
run_qpe(num_count_qubits=3, theta=0.375)

print("\n=== TASK 4: Inverse QFT Visualization ===")
run_qpe(num_count_qubits=3, theta=0.5, visualize=True)

print("\n=== TASK 5: Noise Simulation ===")
run_qpe(num_count_qubits=3, theta=0.125, noisy=True)



=== TASK 1: Change Phase Value ===
     ┌───┐                           ┌───┐                              ┌─┐   »
q_0: ┤ H ├─■───────────────────────X─┤ H ├─■─────────■──────────────────┤M├───»
     ├───┤ │               ┌───┐   │ └───┘ │P(-π/2)  │                  └╥┘┌─┐»
q_1: ┤ H ├─┼────────■──────┤ H ├───┼───────■─────────┼─────────■─────────╫─┤M├»
     ├───┤ │        │      └───┘   │ ┌───┐           │P(-π/4)  │P(-π/2)  ║ └╥┘»
q_2: ┤ H ├─┼────────┼──────■───────X─┤ H ├───────────■─────────■─────────╫──╫─»
     ├───┤ │P(π/2)  │P(π)  │P(2π)    └───┘                               ║  ║ »
q_3: ┤ X ├─■────────■──────■─────────────────────────────────────────────╫──╫─»
     └───┘                                                               ║  ║ »
c: 3/════════════════════════════════════════════════════════════════════╩══╩═»
                                                                         0  1 »
«        
«q_0: ───
«        
«q_1: ───
«     ┌─┐
«q_2: ┤M├
«     └╥┘
«q_3: ─╫─
«   

{'101': 127,
 '100': 18,
 '110': 10,
 '010': 33,
 '000': 60,
 '001': 836,
 '111': 163,
 '011': 801}